# Importação de bibliotecas

In [1]:
!pip install transformers
!pip install pandas
!pip install plotly
!pip install numpy
!pip install tensorflow
!pip install scikit-learn
!pip install keras
!pip install keras-nlp


import pandas as pd
import plotly.express as px
import numpy as np
import re
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score
from transformers import BertTokenizer, TFBertModel, TFBertForSequenceClassification
import keras
import keras_nlp
from keras.layers import Input, Dense, Concatenate
from keras.models import Model

KeyboardInterrupt: 

# Importação dos dados

In [2]:
df = pd.read_csv('/content/bot_detection_data.csv')

In [3]:
df

,User ID,Username,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Location,Created At,Hashtags
0,132131,flong,Station activity person against natural majori...,85,1,2353,False,1,Adkinston,2020-05-11 15:29:50,NaN
1,289683,hinesstephanie,Authority research natural life material staff...,55,5,9617,True,0,Sanderston,2022-11-26 05:18:10,both live
2,779715,roberttran,Manage whose quickly especially foot none to g...,6,2,4363,True,0,Harrisonfurt,2022-08-08 03:16:54,phone ahead
3,696168,pmason,Just cover eight opportunity strong policy which.,54,5,2242,True,1,Martinezberg,2021-08-14 22:27:05,ever quickly new I
4,704441,noah87,Animal sign six data good or.,26,3,8438,False,1,Camachoville,2020-04-13 21:24:21,foreign mention
...,...,...,...,...,...,...,...,...,...,...,...
49995,491196,uberg,Want but put card direction know miss former h...,64,0,9911,True,1,Lake Kimberlyburgh,2023-04-20 11:06:26,teach quality ten education any
49996,739297,jessicamunoz,Provide whole maybe agree church respond most ...,18,5,9900,False,1,Greenbury,2022-10-18 03:57:35,add walk among believe
49997,674475,lynncunningham,Bring different everyone international capital...,43,3,6313,True,1,Deborahfort,2020-07-08 03:54:08,onto admit artist first
49998,167081,richardthompson,Than about single generation itself seek sell ...,45,1,6343,False,0,Stephenside,2022-03-22 12:13:44,star


# Exploração dos dados

Primeiramente vale mencionar que esse dataset é focado na identificação de contas com bot no twitter. Trazendo diversas informações sobre a conta para essa análise além da coluna 'Bot Label', sendo essa a coluna target do modelo a ser construída.

A seguir está uma pequena descrição dos dados a qual e fornecida no Kaggle:


* Username: O nome de usuário associado ao usuário.
* Tweet: O conteúdo de texto do tweet.
* Retweet Count: O número de vezes que o tweet foi retweetado.
* Mention Count: O número de menções no tweet.
* Follower Count: O número de seguidores que o usuário possui.
* Verified: Um valor booleano que indica se o usuário é verificado ou não.
* Bot Label: Um rótulo que indica se o usuário é um bot (1) ou não (0).
* Location: A localização associada ao usuário.
* Created At: A data e hora em que o tweet foi criado.
* Hashtags: As hashtags associadas ao tweet.

Primeiramente vale a pena realizar a analise da classe alvo:

In [4]:
fig = px.histogram(df, x='Bot Label')
fig.show()


Como é possível observar a classe alvo já está balanceada, tendo o mesmo número de bots e não bots. Não sendo necessário um tratamento para isso.

Outro fator importante de se observar e o tipo dos dados de cada coluna, de forma a ter um norte de quais tratamentos realizar.

In [5]:
df.dtypes

,0
User ID,int64
Username,object
Tweet,object
Retweet Count,int64
Mention Count,int64
Follower Count,int64
Verified,bool
Bot Label,int64
Location,object
Created At,object


A partir dessa informações e da descrição fornecida previamente pelo Kaggle e possivel obter algumas informações.

Primeiramente vale mencionar que as colunas User ID	e Username, se referem à informações pessoais do usuario, não sendo adequado utilizar elas para o treinamento do modelo.

In [6]:
print(df[['User ID', 'Username']].head())


   User ID        Username
0   132131           flong
1   289683  hinesstephanie
2   779715      roberttran
3   696168          pmason
4   704441          noah87


Vale mencionar também as colunas que devem passar pelo processo de embadding, que será realizado com o BERT. As quais foi possível observar que são as colunas hashtag e Tweet.


In [7]:
print(df[['Hashtags', 'Tweet']])


                              Hashtags  \
0                                  NaN   
1                            both live   
2                          phone ahead   
3                   ever quickly new I   
4                      foreign mention   
...                                ...   
49995  teach quality ten education any   
49996           add walk among believe   
49997          onto admit artist first   
49998                             star   
49999                             home   

                                                   Tweet  
0      Station activity person against natural majori...  
1      Authority research natural life material staff...  
2      Manage whose quickly especially foot none to g...  
3      Just cover eight opportunity strong policy which.  
4                          Animal sign six data good or.  
...                                                  ...  
49995  Want but put card direction know miss former h...  
49996  Provide whole ma

Outra observação interessante de se realizar e em relação às colunas numéricas do dataset, exceto a label, as quais podem passar pelo processo de normalização, a seguir é possível visualizar as colunas que devem receber esse tratamento:

In [8]:
fig = px.histogram(df, x="Retweet Count", nbins=50, title="Distribuição de Valores da Coluna Retweet Count")
fig.show()


In [9]:
fig = px.histogram(df, x="Mention Count", nbins=50, title="Distribuição de Valores da Coluna Mention Count")
fig.show()


In [10]:
fig = px.histogram(df, x="Follower Count", nbins=50, title="Distribuição de Valores da Coluna Follower Count")
fig.show()


# Tratamentos de dados

## Remoção de colunas a não serem utilizadas


Abaixo serão removidas as colunas que não serão utilizadas para o treinamento do modelo, selecionadas com base na análise preliminar realizada:

In [11]:
df = df.drop(columns=['User ID', 'Username', 'Location', 'Created At','Hashtags'])
df

,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label
0,Station activity person against natural majori...,85,1,2353,False,1
1,Authority research natural life material staff...,55,5,9617,True,0
2,Manage whose quickly especially foot none to g...,6,2,4363,True,0
3,Just cover eight opportunity strong policy which.,54,5,2242,True,1
4,Animal sign six data good or.,26,3,8438,False,1
...,...,...,...,...,...,...
49995,Want but put card direction know miss former h...,64,0,9911,True,1
49996,Provide whole maybe agree church respond most ...,18,5,9900,False,1
49997,Bring different everyone international capital...,43,3,6313,True,1
49998,Than about single generation itself seek sell ...,45,1,6343,False,0


## Normalização

Normalizando dados para uma melhor consistência dos dados e evitar o enviesamento para alguma feature específica:

In [12]:
# Realizando a normalização das colunas numericas dos dados

scaler = MinMaxScaler()
numerical_cols = ['Retweet Count', 'Mention Count', 'Follower Count']

df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

print(df[numerical_cols].describe())


       Retweet Count  Mention Count  Follower Count
count   50000.000000   50000.000000    50000.000000
mean        0.500056       0.502752        0.498860
std         0.291812       0.341713        0.287874
min         0.000000       0.000000        0.000000
25%         0.250000       0.200000        0.248775
50%         0.500000       0.600000        0.499150
75%         0.750000       0.800000        0.747100
max         1.000000       1.000000        1.000000


## Remoção de caracteres especiais

Removendo caracteres especiais para tornar a tokenização mais eficiente

In [13]:
df['Tweet'] = df['Tweet'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))

# Modelo

Primeiramente sera realizada a separação dos dados em treino e teste, sendo destinados 20% dos dados para teste.

In [14]:
# Dividir em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    df['Tweet'], df['Bot Label'], test_size=0.2, random_state=42
)

## Tokenizando dados:

Abaixo será realizada a tokenização dos dados, visando tranforma-los em um formato adequado para o Bert

In [15]:
!pip install keras_nlp
import keras_nlp

## Modelo

Abaixo será definida a estrutura do nosso modelo keras, utilizando do modelo pré treinado do bert para auxiliar nas previsões

In [ ]:
# Converter listas de tokens em arrays numpy
features_train = np.array(X_train.tolist())
features_test = np.array(X_test.tolist())
labels_train = np.array(y_train)
labels_test = np.array(y_test)

# Classificador pré-treinado.
classifier = keras_nlp.models.BertTextClassifier.from_preset(
    "bert_base_en_uncased",
    num_classes=4,
)

# Compilar o modelo
classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(5e-5),
    jit_compile=True,
)

# Treinar o modelo por 10 épocas
classifier.fit(x=features_train, y=labels_train, batch_size=32, epochs=5, steps_per_epoch=32)


# Fazer previsões para o conjunto de treino
predictions_train = classifier.predict(x=features_train, batch_size=2)
predictions_train_labels = np.argmax(predictions_train, axis=1)  # Obter as classes preditas

# Fazer previsões para o conjunto de teste
predictions_test = classifier.predict(x=features_test, batch_size=2)
predictions_test_labels = np.argmax(predictions_test, axis=1)

# Calcular a acurácia e precisão
accuracy_train = accuracy_score(labels_train, predictions_train_labels)
accuracy_test = accuracy_score(labels_test, predictions_test_labels)

precision_train = precision_score(labels_train, predictions_train_labels, average='weighted')
precision_test = precision_score(labels_test, predictions_test_labels, average='weighted')

# Printar os resultados
print(f"Acurácia - Treino: {accuracy_train:.4f}")
print(f"Acurácia - Teste: {accuracy_test:.4f}")
print(f"Precisão - Treino: {precision_train:.4f}")
print(f"Precisão - Teste: {precision_test:.4f}")


Epoch 1/5


Resultado:

Após realizar o treinamento do modelo com base no modelo pré treinado do bert foram obtidos os seguintes resultados:



*   Acuracia treino:
*   Acuracia teste:
*   Precisão treino:
*   Precisão teste:

Assim demonstrando que o modelo Bert foi adequado para indentificação de bots no twitter.

Vale mencionar que por questões de compatibilidade com modelo foi tomada a escolha de se utilizar apenas a coluna ['Tweet'] para analise.



# (Além) adicionar mais features, além da coluna textual

visando conciliar o modelo bert junto com a rede keras, será adiconada as camadas com dados numericos juntamente com o texto a ser proscessado pelo bert

In [ ]:

# Dividir em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    df[['Tweet_tokenized', 'Retweet Count', 'Mention Count', 'Follower Count']],
    df['Bot Label'],
    test_size=0.2,
    random_state=42
)

# Converter listas de tokens em arrays numpy
features_train = np.array(X_train['Tweet_tokenized'].tolist())
features_test = np.array(X_test['Tweet_tokenized'].tolist())

# Colunas numéricas
numeric_features_train = np.array(X_train[['Retweet Count', 'Mention Count', 'Follower Count']])
numeric_features_test = np.array(X_test[['Retweet Count', 'Mention Count', 'Follower Count']])

# Definir o modelo BERT original
bert_model = keras_nlp.models.BertTextClassifier.from_preset(
    "bert_base_en_uncased",
    num_classes=1,  # Para classificação binária
)

# Definir a entrada para as colunas numéricas
numeric_input = Input(shape=(3,), name='numeric_input')

# Combinar a saída do modelo BERT e a entrada numérica
combined_output = Concatenate()([bert_model.output, numeric_input])

# Adicionar uma camada densa após a combinação
dense_layer = Dense(64, activation='relu')(combined_output)
output_layer = Dense(1, activation='sigmoid')(dense_layer)  # Usar sigmoid para binária

# Criar o novo modelo
model = Model(inputs=[bert_model.input, numeric_input], outputs=output_layer)

# Compilar o modelo
model.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=False),  # Usar BinaryCrossentropy para classificação binária
    optimizer=keras.optimizers.Adam(5e-5),
    metrics=['accuracy'],
)

# Treinar o modelo por 5 épocas
model.fit(x=[features_train, numeric_features_train], y=y_train, batch_size=32, epochs=5)

# Fazer previsões para o conjunto de treino
predictions_train = model.predict(x=[features_train, numeric_features_train], batch_size=32)
predictions_train_labels = (predictions_train > 0.5).astype(int)  # Converter para 0 ou 1 com threshold

# Fazer previsões para o conjunto de teste
predictions_test = model.predict(x=[features_test, numeric_features_test], batch_size=32)
predictions_test_labels = (predictions_test > 0.5).astype(int)

# Calcular a acurácia e precisão
accuracy_train = accuracy_score(y_train, predictions_train_labels)
accuracy_test = accuracy_score(y_test, predictions_test_labels)

precision_train = precision_score(y_train, predictions_train_labels)
precision_test = precision_score(y_test, predictions_test_labels)

# Printar os resultados
print(f"Acurácia - Treino: {accuracy_train:.4f}")
print(f"Acurácia - Teste: {accuracy_test:.4f}")
print(f"Precisão - Treino: {precision_train:.4f}")
print(f"Precisão - Teste: {precision_test:.4f}")


Após a adição dos novos dados